In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

# Imports
from pynattas.builder.netBuilder import GenericNetwork 
from pynattas.utils import layerCoder

import configparser
import pynattas as pnas
from datetime import datetime
import time

config = configparser.ConfigParser()
config.read('config.ini')


nas_check = config.getboolean(section='Mode', option='network_architecture_search')
ht_check = config.getboolean(section='Mode', option='hyperparameter_tuning')


task = config['Mode']['task']
if task not in ['C','D']:
    print(f"Error: Selected Task {task} is not available. Check config_optimizer.ini for verification.")
    exit()
else:
    print(f"Task: {task}")

Task: C


In [3]:
# Define NAS parameters and run GA
max_layers = config.getint('NAS', 'max_layers')
max_iterations = int(config['GA']['max_iterations'])
population_size = int(config['GA']['population_size'])
log_path = str(config['GA']['logs_dir_GA'])
mating_pool_cutoff = float(config['GA']['mating_pool_cutoff'])
mutation_probability = float(config['GA']['mutation_probability'])
############################################
print(f"Max Layers: {max_layers}")
print(f"Max Iterations: {max_iterations}")
print(f"Population Size: {population_size}")
print(f"Log Path: {log_path}")
print(f"Mating Pool Cutoff: {mating_pool_cutoff}")
print(f"Mutation Probability: {mutation_probability}")

Max Layers: 10
Max Iterations: 4
Population Size: 8
Log Path: ./logs/GA_logs
Mating Pool Cutoff: 0.8
Mutation Probability: 0.1


In [4]:
# Define population
from pynattas.builder.population import Population

pop = Population(max_layers=max_layers, n_individuals=population_size)
print(f"Initial Population: {len(pop.population)}")
print(f"Current Generation: {pop.iteration}")

[x.architecture for x in pop.population]

Initial Population: 8
Current Generation: 0


['Lne5aln1|PM|Lne6agn1|PI|Lbo4k3s2p2aln1|PI|Ldo4agn1|PM|Leo2k5s1p2aln1|PI|Ldo2aln1|PI|HC||',
 'Leo3k3s1p0agn1|PM|LPe3arn1|PM|Ldo3agn1|PM|LOaln1|PM|Leo3k5s2p0aln1|PM|HC||',
 'Leo4k3s2p1agn1|PM|HC||',
 'LPe4agn1|PM|Lco2k4s2p1aln1|PI|LOarn1|PI|Ldo3arn1|PI|HC||',
 'Lbo2k3s2p0aln1|PM|LRr2agn1|PI|Leo3k5s2p2aln1|PM|Lme4arn1|PI|LOaln1|Pa|Lme3agn1|PM|Leo3k3s1p0aln1|PI|LOagn1|PI|Lbo4k3s2p1agn1|PI|HC||',
 'Lco3k3s1p0agn1|PI|Lne6aln1|Pa|LRr4aln1|PI|Lne3arn1|PI|Leo4k5s1p2agn1|PI|LOarn1|PI|HC||',
 'Lne6aln1|PI|LRr4aln1|Pa|Lme5aln1|Pa|LPe5agn1|PI|HC||',
 'LOarn1|PM|LRr3arn1|Pa|Ldo3arn1|PM|LRr4aln1|PM|Ldo3aln1|Pa|LRr4aln1|PI|LOaln1|Pa|HC||']

In [6]:
idx = 5
parsed_layers = layerCoder.parse_architecture_code(pop.population[idx].architecture)
parsed_layers

[{'layer_type': 'ConvBnAct',
  'out_channels_coefficient': '3',
  'kernel_size': '3',
  'stride': '1',
  'padding': '0',
  'activation': 'GELU',
  'num_blocks': '1'},
 {'layer_type': 'Identity'},
 {'layer_type': 'MBConvNoRes',
  'expansion_factor': '6',
  'activation': 'LeakyReLU',
  'num_blocks': '1'},
 {'layer_type': 'AvgPool'},
 {'layer_type': 'ResNetBlock',
  'reduction_factor': '4',
  'activation': 'LeakyReLU',
  'num_blocks': '1'},
 {'layer_type': 'Identity'},
 {'layer_type': 'MBConvNoRes',
  'expansion_factor': '3',
  'activation': 'ReLU',
  'num_blocks': '1'},
 {'layer_type': 'Identity'},
 {'layer_type': 'ConvSE',
  'out_channels_coefficient': '4',
  'kernel_size': '5',
  'stride': '1',
  'padding': '2',
  'activation': 'GELU',
  'num_blocks': '1'},
 {'layer_type': 'Identity'},
 {'layer_type': 'CSPConvBlock', 'activation': 'ReLU', 'num_blocks': '1'},
 {'layer_type': 'Identity'},
 {'layer_type': 'ClassificationHead'}]

In [7]:
input_channels = 3
input_height = 224
input_width = 224
num_classes = 2

Net = GenericNetwork(
        parsed_layers, 
        input_channels,
        input_height,
        input_width,
        num_classes,
)

In [8]:
Net

GenericNetwork(
  (layers): ModuleList(
    (0): ConvBnAct(
      (conv): Conv2d(32, 96, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): GELU()
    )
    (1): Identity()
    (2): MBConvNoRes(
      (steps): Sequential(
        (0): ConvBnAct(
          (conv): Conv2d(96, 576, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(576, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): LeakyReLU(
            (LeakyReLU): LeakyReLU(negative_slope=0.1)
          )
        )
        (1): Conv2d(576, 576, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=576)
        (2): BatchNorm2d(576, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): LeakyReLU(
          (LeakyReLU): LeakyReLU(negative_slope=0.1)
        )
        (4): ConvBnAct(
          (conv): Conv2d(576, 96, kernel_size=(1, 1), stride=(1, 1), bia

In [41]:
import re

segment = 'Lco3k4s2p1agn1'
##########################
params = segment[2:]  # Parameters portion of the segment

# Extract alphanumeric pairs, keeping in mind cases like 'agn1'
param_pairs = re.findall(r'[a-zA-Z]+[0-9]*', params)

segment_info = {}
for pair in param_pairs:
    # Special case for 'agn1': manually split 'a:g' and 'n:1'
    if pair.startswith('a'):
        key_value_pairs = [(pair[0], pair[1]), (pair[2], pair[3])]
    else:
        key_value_pairs = [(pair[0], pair[1:])]

    segment_info.update({key: value for key, value in key_value_pairs})

In [6]:
# from pynattas.optimizers import ga

In [ ]:
# nas_result = pnas.optimizers.ga.ga_optimizer(
#     max_layers=max_layers,
#     max_iter=max_iterations,
#     n_individuals=population_size,
#     mating_pool_cutoff=mating_pool_cutoff,
#     mutation_probability=mutation_probability,
#     logs_directory=log_path,
#     task=task,
# )